In [97]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

In [99]:
dfCrime = pd.read_csv('/Users/austincoffelt/Downloads/Rent-Walk-Local/Crime_Data_from_2010_to_2019_20250419.csv')
print(dfCrime.shape)
dfCrime.head()

(2123183, 28)


,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,1307355,02/20/2010 12:00:00 AM,02/20/2010 12:00:00 AM,1350,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,...,AA,Adult Arrest,900.0,NaN,NaN,NaN,300 E GAGE AV,NaN,33.9825,-118.2695
1,11401303,09/13/2010 12:00:00 AM,09/12/2010 12:00:00 AM,45,14,Pacific,1485,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,IC,Invest Cont,740.0,NaN,NaN,NaN,SEPULVEDA BL,MANCHESTER AV,33.9599,-118.3962
2,70309629,08/09/2010 12:00:00 AM,08/09/2010 12:00:00 AM,1515,13,Newton,1324,2,946,OTHER MISCELLANEOUS CRIME,...,IC,Invest Cont,946.0,NaN,NaN,NaN,1300 E 21ST ST,NaN,34.0224,-118.2524
3,90631215,01/05/2010 12:00:00 AM,01/05/2010 12:00:00 AM,150,6,Hollywood,646,2,900,VIOLATION OF COURT ORDER,...,IC,Invest Cont,900.0,998.0,NaN,NaN,CAHUENGA BL,HOLLYWOOD BL,34.1016,-118.3295
4,100100501,01/03/2010 12:00:00 AM,01/02/2010 12:00:00 AM,2100,1,Central,176,1,122,"RAPE, ATTEMPTED",...,IC,Invest Cont,122.0,NaN,NaN,NaN,8TH ST,SAN PEDRO ST,34.0387,-118.2488


In [100]:
violentCrimeCodes = [110, 113, 121, 122, 210, 220, 230, 231]
dfCrime = dfCrime[dfCrime['Crm Cd'].isin(violentCrimeCodes)]
print(dfCrime.shape)
dfCrime.head()

(205005, 28)


,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
4,100100501,01/03/2010 12:00:00 AM,01/02/2010 12:00:00 AM,2100,1,Central,176,1,122,"RAPE, ATTEMPTED",...,IC,Invest Cont,122.0,NaN,NaN,NaN,8TH ST,SAN PEDRO ST,34.0387,-118.2488
7,100100509,01/09/2010 12:00:00 AM,01/08/2010 12:00:00 AM,2100,1,Central,157,1,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",...,AA,Adult Arrest,230.0,NaN,NaN,NaN,500 CROCKER ST,NaN,34.0435,-118.2427
8,100100510,01/09/2010 12:00:00 AM,01/09/2010 12:00:00 AM,230,1,Central,171,1,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",...,IC,Invest Cont,230.0,NaN,NaN,NaN,800 W OLYMPIC BL,NaN,34.0450,-118.2640
11,100100522,01/15/2010 12:00:00 AM,01/14/2010 12:00:00 AM,2000,1,Central,158,1,210,ROBBERY,...,AO,Adult Other,210.0,NaN,NaN,NaN,ALAMEDA ST,7TH ST,34.0350,-118.2386
14,100100531,01/16/2010 12:00:00 AM,01/15/2010 12:00:00 AM,2030,1,Central,127,1,210,ROBBERY,...,IC,Invest Cont,210.0,NaN,NaN,NaN,1ST,LOS ANGELES,34.0515,-118.2424


In [101]:
dfCrime = dfCrime[['Crm Cd', 'DATE OCC',  'LON', 'LAT']]
dfCrime.head()

,Crm Cd,DATE OCC,LON,LAT
4,122,01/02/2010 12:00:00 AM,-118.2488,34.0387
7,230,01/08/2010 12:00:00 AM,-118.2427,34.0435
8,230,01/09/2010 12:00:00 AM,-118.2640,34.0450
11,210,01/14/2010 12:00:00 AM,-118.2386,34.0350
14,210,01/15/2010 12:00:00 AM,-118.2424,34.0515


In [102]:
# make our lon and lat into a point type for spacial join
dfCrime['geometry'] = dfCrime.apply(lambda row: Point(row['LON'], row['LAT']), axis=1)
dfCrime = dfCrime[['Crm Cd', 'DATE OCC', 'geometry']]
print(dfCrime.shape)
dfCrime.head()

(205005, 3)


,Crm Cd,DATE OCC,geometry
4,122,01/02/2010 12:00:00 AM,POINT (-118.2488 34.0387)
7,230,01/08/2010 12:00:00 AM,POINT (-118.2427 34.0435)
8,230,01/09/2010 12:00:00 AM,POINT (-118.264 34.045)
11,210,01/14/2010 12:00:00 AM,POINT (-118.2386 34.035)
14,210,01/15/2010 12:00:00 AM,POINT (-118.2424 34.0515)


In [103]:
dfCrime['DATE OCC'] = pd.to_datetime(dfCrime['DATE OCC'])
dfCrime18 = dfCrime[(dfCrime['DATE OCC'] >= '2018-01-01') & (dfCrime['DATE OCC'] < '2019-01-01')]
dfCrime17 = dfCrime[(dfCrime['DATE OCC'] >= '2017-01-01') & (dfCrime['DATE OCC'] < '2018-01-01')]
dfCrime16 = dfCrime[(dfCrime['DATE OCC'] >= '2016-01-01') & (dfCrime['DATE OCC'] < '2017-01-01')]
dfCrime15 = dfCrime[(dfCrime['DATE OCC'] >= '2015-01-01') & (dfCrime['DATE OCC'] < '2016-01-01')]
dfCrime14 = dfCrime[(dfCrime['DATE OCC'] >= '2014-01-01') & (dfCrime['DATE OCC'] < '2015-01-01')]

/var/folders/6z/0pv81j2524v8c80qsmnrbmcm0000gn/T/ipykernel_12383/1966438854.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dfCrime['DATE OCC'] = pd.to_datetime(dfCrime['DATE OCC'])


In [104]:
# Read shapefile (automatically loads .shp, .shx, .dbf)
gdf = gpd.read_file("/Users/austincoffelt/Downloads/Rent-Walk-Local/cb_2018_06_bg_500k/cb_2018_06_bg_500k.shp")

# View the first 5 rows
print(gdf.shape)
gdf.head()

(23196, 11)


,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,06,075,980401,1,1500000US060759804011,060759804011,1,BG,419323,247501289,"POLYGON ((-123.01 37.7, -123.01 37.699, -123.0..."
1,06,059,062702,1,1500000US060590627021,060590627021,1,BG,513205,635569,"POLYGON ((-117.88 33.593, -117.88 33.595, -117..."
2,06,037,576602,1,1500000US060375766021,060375766021,1,BG,246841,1907927,"MULTIPOLYGON (((-118.18 33.765, -118.17 33.765..."
3,06,075,061500,6,1500000US060750615006,060750615006,6,BG,313325,229501,"POLYGON ((-122.39 37.783, -122.39 37.787, -122..."
4,06,049,000300,2,1500000US060490003002,060490003002,2,BG,1781768492,310119189,"POLYGON ((-120.82 41.994, -120.79 41.994, -120..."


In [105]:
# make sure points match the crs of gdf
gdfPoints18 = gpd.GeoDataFrame(
    dfCrime18, 
    geometry='geometry',
    crs="EPSG:4269"  # WGS84 coordinate reference system
)
gdfPoints18.head()

,Crm Cd,DATE OCC,geometry
1674559,230,2018-03-03,POINT (-118.29 33.979)
1674576,230,2018-08-17,POINT (-118.28 33.79)
1674628,230,2018-05-20,POINT (-118.29 33.955)
1674631,210,2018-02-04,POINT (-118.37 34.046)
1674647,210,2018-09-24,POINT (-118.47 34.228)


In [106]:
# Basic spatial join (points within polygons)
result18 = gpd.sjoin(
    gdfPoints18, 
    gdf,
    how="left",       # 'left', 'right', or 'inner'
    predicate="within" # Spatial relationship
)
print(result18.shape)
result18.head()

(22857, 14)


,Crm Cd,DATE OCC,geometry,index_right,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER
1674559,230,2018-03-03,POINT (-118.29 33.979),18578.0,06,037,237401,2,1500000US060372374012,060372374012,2,BG,168512.0,0.0
1674576,230,2018-08-17,POINT (-118.28 33.79),16856.0,06,037,294302,2,1500000US060372943022,060372943022,2,BG,551098.0,0.0
1674628,230,2018-05-20,POINT (-118.29 33.955),10012.0,06,037,240300,1,1500000US060372403001,060372403001,1,BG,236496.0,0.0
1674631,210,2018-02-04,POINT (-118.37 34.046),18510.0,06,037,216900,5,1500000US060372169005,060372169005,5,BG,189159.0,0.0
1674647,210,2018-09-24,POINT (-118.47 34.228),2694.0,06,037,117407,1,1500000US060371174071,060371174071,1,BG,198328.0,0.0


In [110]:
crime18 = result18['AFFGEOID'].value_counts()
crime18

AFFGEOID
1500000US060372077101    231
1500000US060372260021    182
1500000US060372063003    162
1500000US060371907001    156
1500000US060372063001    148
                        ... 
1500000US060372011101      1
1500000US060372283104      1
1500000US060377006004      1
1500000US060372640001      1
1500000US060372628022      1
Name: count, Length: 2291, dtype: int64

In [117]:
# export as a csv
crime18.to_csv('LA_Violent_Crime_BG.csv')